# Verification of Iteration 0 Results

This notebook verifies the input distributions and physics behavior for iteration 0 simulations of a specific geometry (regular spheres, irregular spheres, or realistic grains) in g4chargeit.

## Setup Requirements:
1. Run the iteration 0 simulation with 1 million particles
2. ROOT files should be in `../build/root/`
3. Required packages: uproot, matplotlib, numpy, pandas, scipy, trimesh

In [ ]:
# Import required libraries
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.constants import epsilon_0, e as q_e
from matplotlib.colors import LogNorm, Normalize
import glob
import os
import trimesh
from trimesh.points import PointCloud
from shapely.geometry import Polygon as ShapelyPolygon
from matplotlib.patches import Polygon

from scipy.interpolate import interp1d

from common_functions import *

print("Libraries imported successfully!")

In [ ]:
geometry = trimesh.load_mesh('../g4chargeit/geometry/irregularSpheres_fromPython.stl')
# other options: irregularSpheres_fromPython.stl, realisticGrains_fromPython.stl, regularSpheres_radius100um_fromPython.stl
print(f"Geometry loaded: {len(geometry.faces)} faces")
geometry.show()

## 1. Load Iteration 0 Results

Choose which configuration to analyze:
- `"onlyphotoemission"`: Photoemission (only incident photons)
- `"onlysolarwind"`: Solar Wind (only incident protons and low-energy e-)
- `"only*"`: reads in ROOT file for both cases

In [ ]:
# Configuration: choose one
config = "onlysolarwind*"  # Options: "onlyphotoemission", "onlysolarwind", "only"

# Define directory and file list
directory_path = "../build-regular-1millionSW/root/"
filelist = sorted(glob.glob(f"{directory_path}/*iteration0*{config}*1000000.root"))

if len(filelist) == 0:
    print(f"WARNING: No files found matching pattern: *iteration0*{config}.root")
    print(f"Please check that you have run the simulation and files are in {directory_path}")
else:
    print(f"Found {len(filelist)} file(s) to process:")
    for f in filelist:
        print(f"  - {os.path.basename(f)}")

In [ ]:
for fileIN in filelist:
    filename = os.path.basename(fileIN)
    print(f"\nProcessing: {filename}")
    
    # Extract iteration number
    number_str = filename.split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))
    
    # Extract config from filename
    config_from_file = filename.split("_")[2]
    
    # Read data and calculate statistics
    vars()[f"df_{config_from_file}"] = read_rootfile(filename, directory_path=directory_path)
    _ = calculate_stats(vars()[f"df_{config_from_file}"], config=config_from_file, printout=True)
    
    print("-" * 78)

## 2. Solar Wind Verification

Verify input distributions for solar wind electrons and protons.

In [ ]:
print(f"Read in original differential flux distributions ...")

# Read in the original differential SW ions flux distribution
input_dist = pd.read_csv("../g4chargeit/distributions/Fig4-Li2023-SWions.csv")
x_data = np.array(input_dist["x"])     # Wavelength [nm]
y_data = np.array(input_dist[" y"])    # Differential photon flux
print(f"Ions data range: {min(x_data):.2f} to {max(x_data):.2f} eV")

# interpolate the distribution
ions_inputDistribution_energies = np.linspace(min(x_data), max(x_data), 200)
interp_func = interp1d(x_data,y_data,kind='linear',bounds_error=False,fill_value='extrapolate')
ions_binWidths = np.diff(ions_inputDistribution_energies)
ions_inputDistribution_yValues = interp_func(ions_inputDistribution_energies)
ions_currentDensity = 1.602e-19 * np.sum(ions_inputDistribution_yValues[:-1] * ions_binWidths * 1e4)

# Read in the original differential SW electrons flux distribution
input_dist = pd.read_csv("../g4chargeit/distributions/Fig4-Li2023-SWelectrons.csv")
x_data = np.array(input_dist["x"])     # Wavelength [nm]
y_data = np.array(input_dist[" y"])    # Differential photon flux
print(f"Electrons data range: {min(x_data):.2f} to {max(x_data):.2f} eV\n")

# interpolate the distribution
electrons_inputDistribution_energies = np.logspace(0, 4, 500)
interp_func = interp1d(x_data,y_data,kind='linear',bounds_error=False,fill_value='extrapolate')
electrons_binWidths = np.diff(electrons_inputDistribution_energies)
electrons_inputDistribution_yValues = interp_func(electrons_inputDistribution_energies)
electrons_currentDensity = 1.602e-19 * np.sum(electrons_inputDistribution_yValues[:-1] * electrons_binWidths * 1e4)

# ------------------------------------------------------------------
# Plot the interpolated differential flux spectrum
# ------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(8, 5))

ax.loglog(electrons_inputDistribution_energies,electrons_inputDistribution_yValues,'r.-',linewidth=2,label='SW e-')
ax.loglog(ions_inputDistribution_energies,ions_inputDistribution_yValues,'b.-',linewidth=2,label='SW ions')

# ------------------------------------------------------------------
# calculate current density of SW ions and electrons current density
# ------------------------------------------------------------------

print(f"Calculate Current Densities ...")

print(f"SW e-: {electrons_currentDensity:.5e} A/m2")
print(f"SW ions: {ions_currentDensity:.5e} A/m2")
print(f"Factor of e-/ions: {electrons_currentDensity/ions_currentDensity:.5f}")

# ax.set_xlim([6, 330])
# ax.set_ylim([1e-4, 1])

ax.set_xlabel("Energy (eV)")
ax.set_ylabel("Differential Flux (s$^{-1}$ cm$^{-2}$ sr$^{-1}$ eV$^{-1}$)")
ax.axhline(y=7e2,color="k")
ax.axhline(y=2e6,color="k")
ax.axhline(y=7e9,color="k")
plt.show()


In [ ]:
import matplotlib as mpl
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 14})

# Check electron energy distribution
if 'df_onlysolarwind' not in dir():
    print("Solar wind data not loaded. Set config='onlysolarwind' and rerun to analyze.")
else:
    fig, ax = plt.subplots(figsize=(6.8, 3))
    sw_electrons = df_onlysolarwind[df_onlysolarwind["Particle_Type"] == "e-"].drop_duplicates(
        subset="Event_Number", keep="first"
    )
    histdata_e = ax.hist(
        sw_electrons["Kinetic_Energy_Pre_MeV"] * 1e6,
        bins=np.logspace(0,4,500),
        density=True,
        alpha=0.6,
        label='Simulation'
    )
    print(f"avg e- energy: {np.mean(sw_electrons['Kinetic_Energy_Pre_MeV']) * 1e6} eV")
    print(f"max e- energy: {np.max(sw_electrons['Kinetic_Energy_Pre_MeV']) * 1e6} eV")

    ax.loglog(electrons_inputDistribution_energies,\
        electrons_inputDistribution_yValues / np.max(electrons_inputDistribution_yValues) * max(histdata_e[0]),\
        'b-',linewidth=2,label='SW e-'
    )

    sw_ions = df_onlysolarwind[df_onlysolarwind["Particle_Type"] == "proton"].drop_duplicates(
        subset="Event_Number", keep="first"
    )
    histdata = ax.hist(
        sw_ions["Kinetic_Energy_Pre_MeV"] * 1e6,
        bins=np.linspace(150,50000,500),
        density=True,
        alpha=0.6,color="pink",
        label='Simulation'
    )
    print(f"avg ion energy: {np.mean(sw_ions['Kinetic_Energy_Pre_MeV']) * 1000} keV")

    ax.loglog(ions_inputDistribution_energies[8:],\
        ions_inputDistribution_yValues[8:] / np.max(ions_inputDistribution_yValues) * max(histdata[0][10:]),\
        'r-',linewidth=2,label='SW ions'
    )
    
    ax.set_xlim([1,6e3])
    ax.set_ylim(10e-8,8)
    ax.set_xlabel("Energy (eV)")
    ax.set_ylabel("Probability Density")
    ax.set_xscale("log")
    ax.set_yscale("log")
    plt.tight_layout()


    # ax.axhline(y=max(histdata_e[0])-7e9-2e6-7e2,color="k")
    # ax.axhline(y= max(histdata_e[0])-7e9-2e6,color="r")
    # ax.axhline(y= max(histdata_e[0]),color="k")
    # ax.axhline(y= max(histdata[0]),color="k")

    print(f"factor: {max(histdata_e[0])/max(histdata[0])}, desired: {7e9/2e6/2/np.pi}")

    plt.show()

In [ ]:
# Check electron energy distribution
if 'df_onlysolarwind' not in dir():
    print("Solar wind data not loaded. Set config='onlysolarwind' and rerun to analyze.")
else:
    v, vth = np.linspace(0, 50, 100), 10
    electron_distribution = (v**2 / (vth * np.sqrt(np.pi))) * np.exp(-v**2 / vth**2)
    
    fig, ax = plt.subplots(figsize=(8, 5))
    sw_electrons = df_onlysolarwind[df_onlysolarwind["Particle_Type"] == "e-"].drop_duplicates(
        subset="Event_Number", keep="first"
    )
    histdata = ax.hist(
        sw_electrons["Kinetic_Energy_Pre_MeV"] * 1e6,
        bins=100,
        density=True,
        alpha=0.6,
        label='Simulation'
    )
    ax.plot(
        v,
        electron_distribution / np.max(electron_distribution) * max(histdata[0][10:]),
        "r-",
        linewidth=2,
        label='Expected distribution'
    )
    ax.set_xlim([-2, 30])
    ax.set_xlabel("Electron Kinetic Energy (eV)")
    ax.set_ylabel("Probability Density")
    ax.set_title("Solar Wind Electron Energy Distribution")
    ax.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize particle stopping locations
if 'df_onlysolarwind' in dir():
    last_protons = df_onlysolarwind[
        df_onlysolarwind["Particle_Type"] == "proton"
    ].drop_duplicates(subset="Event_Number", keep="last")
    protons_inside = last_protons[last_protons["Volume_Name_Post"] == "SiO2"][::100]
    
    last_electrons = df_onlysolarwind[
        df_onlysolarwind["Particle_Type"] == "e-"
    ].drop_duplicates(subset="Event_Number", keep="last")
    electrons_inside = last_electrons[last_electrons["Volume_Name_Post"] == "SiO2"][::100]
    
    print(f"Protons stopped inside: {len(last_protons[last_protons['Volume_Name_Post'] == 'SiO2'])}")
    print(f"Electrons stopped inside: {len(last_electrons[last_electrons['Volume_Name_Post'] == 'SiO2'])}")
    
    # Create point clouds
    proton_points = PointCloud(
        np.array(protons_inside["Pre_Step_Position_mm"].tolist()),
        colors=[0, 255, 0, 255]  # Green
    )
    electron_points = PointCloud(
        np.array(electrons_inside["Pre_Step_Position_mm"].tolist()),
        colors=[0, 0, 255, 255]  # Blue
    )
    
scene = trimesh.Scene([geometry, proton_points, electron_points])
scene.show()

In [ ]:
# Face illumination for incident protons
if 'df_onlysolarwind' in dir():
    print("Face Illumination Analysis - Incident Protons")
    incident_protons = df_onlysolarwind[
        (df_onlysolarwind["Particle_Type"] == "proton") & 
        (df_onlysolarwind["Parent_ID"] == 0.0) & 
        (df_onlysolarwind["Volume_Name_Post"] == "SiO2")
    ].drop_duplicates(subset="Event_Number", keep="first")
    
    surface, face_totals, _ = plot_face_illumination(
        incident_protons, geometry, vmin=0, vmax=20
    )
    print(f"Face illumination range: {min(face_totals):.2f} to {max(face_totals):.2f}")
    
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.show()

In [ ]:
# all incident protons are at 1 keV
plt.hist(incident_protons["Kinetic_Energy_Pre_MeV"]*1000, 10, color="lightblue",density=True)
plt.xlim([0.99,1.01])
plt.show()

In [ ]:
# Face illumination for incident electrons
if 'df_onlysolarwind' in dir():
    print("Face Illumination Analysis - Incident Electrons")
    incident_electrons = df_onlysolarwind[
        (df_onlysolarwind["Particle_Type"] == "e-") & 
        (df_onlysolarwind["Parent_ID"] == 0.0) &
        (df_onlysolarwind["Volume_Name_Post"] == "SiO2")
    ].drop_duplicates(subset="Event_Number", keep="first")
    
    surface, face_totals, _ = plot_face_illumination(
        incident_electrons, geometry, vmin=0, vmax=90
    )
    print(f"Face illumination range: {min(face_totals):.2f} to {max(face_totals):.2f}")
    
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.show()

Verify the implanation depth of the protons and electrons.

In [ ]:
proton_events = df_onlysolarwind[df_onlysolarwind["Particle_Type"]=="proton"]

# Extract positions into arrays
pre = np.vstack(proton_events["Pre_Step_Position_mm"].to_numpy())
post = np.vstack(proton_events["Post_Step_Position_mm"].to_numpy())

# Compute per-row distances (vectorized)
distances = np.linalg.norm(post - pre, axis=1)* 1e6  # mm → nm

# Add distances into dataframe
proton_events = proton_events.assign(Distance_nm = distances)

# move all proton events that end up leaving the material 
escaping_events = (
    proton_events
    .loc[proton_events["Volume_Name_Post"] == "physical_cyclic", "Event_Number"]
    .unique()
)
proton_events_stopped = proton_events[~proton_events["Event_Number"].isin(escaping_events)]

# Sum distances per Event_Number
distance_per_event_nm = (
    proton_events_stopped.groupby("Event_Number")["Distance_nm"].sum() 
)

plt.hist(distance_per_event_nm,bins=90)
print(f"Averge implanation depth of protons: {np.mean(distance_per_event_nm)} nm")
plt.show()

In [ ]:
electron_events = df_onlysolarwind[df_onlysolarwind["Particle_Type"]=="e-"]

# Extract positions into arrays
pre = np.vstack(electron_events["Pre_Step_Position_mm"].to_numpy())
post = np.vstack(electron_events["Post_Step_Position_mm"].to_numpy())

# Compute per-row distances (vectorized)
distances = np.linalg.norm(post - pre, axis=1)* 1e6  # mm → nm

# Add distances into dataframe
electron_events = electron_events.assign(Distance_nm = distances)

# move all proton events that end up leaving the material 
escaping_events = (
    electron_events
    .loc[electron_events["Volume_Name_Post"] == "physical_cyclic", "Event_Number"]
    .unique()
)
electrons_events_stopped = electron_events[~electron_events["Event_Number"].isin(escaping_events)]

# Sum distances per Event_Number
distance_per_event_nm = (
    electrons_events_stopped.groupby("Event_Number")["Distance_nm"].sum() 
)

plt.hist(distance_per_event_nm,bins=90)
print(f"Averge implanation depth of e-: {np.mean(distance_per_event_nm)} nm")
plt.show()

## 3. Photoemission Verification

Verify photon input distribution.

In [ ]:
from scipy.interpolate import interp1d

# ------------------------------------------------------------------
# Read in the original differential photon flux distribution
# Expected format:
#   x : wavelength in nm
#   y : differential flux in photons / (cm^2 · s · eV)
# ------------------------------------------------------------------
input_dist = pd.read_csv("../g4chargeit/distributions/Fig2-FarrellSolarMinimum.csv")

x_data = np.array(input_dist["x"])     # Wavelength [nm]
y_data = np.array(input_dist[" y"])    # Differential photon flux

# ------------------------------------------------------------------
# Convert wavelength (nm) to photon energy (eV)
# E[eV] = 1240 / λ[nm]
# ------------------------------------------------------------------
x_transform = lambda wavelength_nm: 1240 / wavelength_nm
x_data = x_transform(x_data)

print(f"Data range: {min(x_data):.2f} to {max(x_data):.2f} eV")

# ------------------------------------------------------------------
# Interpolate the spectrum onto a uniform energy grid
# Energy range: 8.1–330 eV
# ------------------------------------------------------------------
inputDistribution_energies = np.linspace(8.1, 330, 1000)

interp_func = interp1d(
    x_data,
    y_data,
    kind='linear',
    bounds_error=False,
    fill_value='extrapolate'
)

# Bin widths in energy (not currently used in the integration)
bin_widths = np.diff(inputDistribution_energies)

# Interpolated differential photon flux
inputDistribution_yValues = interp_func(inputDistribution_energies)

# ------------------------------------------------------------------
# Plot the interpolated differential flux spectrum
# ------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(
    inputDistribution_energies,
    inputDistribution_yValues,
    'r-',
    linewidth=2,
    label='Expected distribution'
)

# ------------------------------------------------------------------
# Estimate photon-generated current density
# Assumptions:
#   - photoemission efficiency => 0.88 electron generated per photon
#   - Flux converted from cm^-2 to m^-2 (× 1e4)
# ------------------------------------------------------------------
PE_efficiency = 0.88*0.01 # 1% of the generated PE escape the material
current_density = PE_efficiency* 1.602e-19 * np.sum( #1.602e-19 *
    inputDistribution_yValues[:-1] * bin_widths * 1e4
)
print(f"Current Density of PE: {current_density:.5e} A/m2")

# ax.set_xlim([6, 330])
# ax.set_ylim([1e-4, 1])

ax.set_xlabel("Photon Energy (eV)")
ax.set_ylabel("Differential Flux (photons cm$^{-2}$ s$^{-1}$ eV$^{-1}$)")
plt.show()


In [ ]:
# Check photon energy distribution
if 'df_onlyphotoemission' not in dir():
    print("Photoemission data not loaded. Set config='onlyphotoemission' and rerun to analyze.")
else:
    input_dist = pd.read_csv(
        "../g4chargeit/distributions/photonSolar_distribution.txt",
        header=None,
        delimiter=" "
    )

    
    incident_gamma = df_onlyphotoemission[
        (df_onlyphotoemission["Particle_Type"] == "gamma") &
        (df_onlyphotoemission["Parent_ID"] == 0.0)
    ].drop_duplicates(subset="Event_Number", keep="first")
    
    fig, ax = plt.subplots(figsize=(8, 5))
    histdata = ax.hist(
        incident_gamma["Kinetic_Energy_Pre_MeV"] * 1e6,
        bins=np.logspace(0.8, 3.1, 100),
        density=True,
        alpha=0.5,
        color='k',
        label='Simulation'
    )
    ax.loglog(
        input_dist[0] * 1e6,
        input_dist[1] / np.max(histdata[0]) * 2.0,
        'r-',
        linewidth=2,
        label='Expected distribution'
    )
    ax.set_xlim([6, 330])
    ax.set_ylim([1e-4, 1])
    ax.set_xlabel("Photon Energy (eV)")
    ax.set_ylabel("Differential Flux")
    ax.set_title("Solar Photon Energy Distribution")
    ax.legend()
    print(f"Number of incident photons: {len(incident_gamma)}")
    plt.tight_layout()
    plt.show()

Verify the implanation depth of the photons.

In [ ]:
gamma_events = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"]=="gamma"]

# Extract positions into arrays
pre = np.vstack(gamma_events["Pre_Step_Position_mm"].to_numpy())
post = np.vstack(gamma_events["Post_Step_Position_mm"].to_numpy())

# Compute per-row distances (vectorized)
distances = np.linalg.norm(post - pre, axis=1)* 1e6  # mm → nm

# Add distances into dataframe
gamma_events = gamma_events.assign(Distance_nm = distances)

# move all proton events that end up leaving the material 
escaping_events = (
    gamma_events
    .loc[gamma_events["Volume_Name_Post"] == "physical_cyclic", "Event_Number"]
    .unique()
)
gamma_distance = gamma_events[~gamma_events["Event_Number"].isin(escaping_events)]

# Sum distances per Event_Number
distance_per_event_nm = (
    gamma_distance.groupby("Event_Number")["Distance_nm"].sum() 
)

plt.hist(distance_per_event_nm,bins=90)
print(f"Averge implanation depth of gamma: {np.mean(distance_per_event_nm)} nm")
print(f"Median implanation depth of gamma: {np.median(distance_per_event_nm)} nm")
plt.show()

Plot distributions of photoemission.

In [ ]:
df_electrons = df_onlyphotoemission[
    df_onlyphotoemission["Particle_Type"] == "e-"
] #.drop_duplicates(subset="Event_Number", keep="last")

photoelectron_creation = df_onlyphotoemission[
    df_onlyphotoemission["Particle_Type"] == "e-"
].drop_duplicates(subset="Event_Number", keep="first")

last_e_event = df_onlyphotoemission[
    df_onlyphotoemission["Particle_Type"] == "e-"
].drop_duplicates(subset="Event_Number", keep="last")

world_e_energy = last_e_event[
    (last_e_event["Volume_Name_Post"] == "physical_cyclic") | 
    (last_e_event["Volume_Name_Pre"] == "physical_cyclic")
]

events_SiO2 = df_electrons[df_electrons["Volume_Name_Post"]=="physical_cyclic"].drop_duplicates(subset="Event_Number", keep="last")

In [ ]:
len(world_e_energy)

In [ ]:
events_SiO2 = photoelectron_creation[photoelectron_creation["Volume_Name_Post"]=="physical_cyclic"].drop_duplicates(subset="Event_Number", keep="last")

In [ ]:
print(len(world_e_energy)/len(photoelectron_creation),len(events_SiO2)/len(photoelectron_creation),len(photoelectron_creation))

In [ ]:
100-

In [ ]:
0.01771284422955644*100

In [ ]:
# Photoelectron energy distribution
if 'df_onlyphotoemission' in dir():
    photoelectron_creation = df_onlyphotoemission[
        df_onlyphotoemission["Particle_Type"] == "e-"
    ].drop_duplicates(subset="Event_Number", keep="first")
    
    last_e_event = df_onlyphotoemission[
        df_onlyphotoemission["Particle_Type"] == "e-"
    ].drop_duplicates(subset="Event_Number", keep="last")
    
    world_e_energy = last_e_event[
        (last_e_event["Volume_Name_Post"] == "physical_cyclic") | 
        (last_e_event["Volume_Name_Pre"] == "physical_cyclic")
    ]
    
    bins = np.logspace(-4, 3.2, 200)
    hist_all, bin_edges = np.histogram(
        photoelectron_creation["Kinetic_Energy_Pre_MeV"] * 1e6,
        bins=bins
    )
    counts_ejected, _ = np.histogram(
        world_e_energy["Kinetic_Energy_Post_MeV"] * 1e6,
        bins=bins
    )
    
    bin_widths = np.diff(bin_edges)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    
    # Convert to differential flux
    differential_allPE = hist_all / bin_widths
    differential_ejected = counts_ejected / bin_widths
    
    # Filter low values
    threshold = 0.1
    diff_all_filtered = np.where(differential_allPE > threshold, differential_allPE, np.nan)
    diff_ejected_filtered = np.where(differential_ejected > threshold, differential_ejected, np.nan)
    
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.step(
        bin_centers, diff_all_filtered,
        where='mid', color='darkblue', alpha=0.7, linewidth=1.5,
        label='All photoelectrons'
    )
    ax.fill_between(
        bin_centers, diff_all_filtered,
        step='mid', color='darkblue', alpha=0.3, linewidth=0
    )
    ax.step(
        bin_centers, diff_ejected_filtered,
        where='mid', color='darkgreen', alpha=0.7, linewidth=1.5,
        label='Emitted e⁻'
    )
    ax.fill_between(
        bin_centers, diff_ejected_filtered,
        step='mid', color='darkgreen', alpha=0.3, linewidth=0
    )
    
    ax.set_xlabel("Photoelectron Energy (eV)")
    ax.set_ylabel("Differential Flux")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlim([bins[0], 330])
    ax.set_title("Photoelectron Energy Distribution")
    ax.legend()
    
    print(f"Average photoelectron energy at creation: {np.mean(photoelectron_creation['Kinetic_Energy_Pre_MeV'] * 1e6):.2f} eV")
    print(f"Total photoelectrons created: {len(photoelectron_creation)}")
    print(f"Photo photoelectron creation efficiency: {len(photoelectron_creation) / len(incident_gamma) * 100:.3f}%")
    print(f"Electrons emitted (reaching boundary): {len(world_e_energy)}")
    print(f"Emission efficiency: {len(world_e_energy) / len(photoelectron_creation) * 100:.3f}%")
    print(f"Average final energy of emitted e⁻: {np.mean(world_e_energy['Kinetic_Energy_Post_MeV'] * 1e6):.2f} eV")
    
    plt.tight_layout()
    plt.show()

### Photoelectric Yield Comparison with Literature

Compare simulation results with published experimental data.

In [ ]:
# Calculate yield per photoelectric absorption event
if 'df_onlyphotoemission' in dir():
    bins = np.logspace(0.8, 3.3, 100)
    
    photoelectron_creation = df_onlyphotoemission[
        df_onlyphotoemission["Particle_Type"] == "e-"
    ].drop_duplicates(subset="Event_Number", keep="first")
    
    last_e_event = df_onlyphotoemission[
        df_onlyphotoemission["Particle_Type"] == "e-"
    ].drop_duplicates(subset=["Parent_ID", "Event_Number"], keep="last")
    
    world_e_energy = last_e_event[
        (last_e_event["Volume_Name_Post"] == "physical_cyclic") | 
        (last_e_event["Volume_Name_Pre"] == "physical_cyclic")
    ]
    
    hist_absorption_events, _ = np.histogram(
        photoelectron_creation["Kinetic_Energy_Pre_MeV"] * 1e6,
        bins=bins
    )
    counts_ejected, _ = np.histogram(
        world_e_energy["Kinetic_Energy_Post_MeV"] * 1e6,
        bins=bins
    )
    
    bin_centers = 0.5 * (bins[1:] + bins[:-1])
    
    yield_per_absorption = np.divide(
        counts_ejected, hist_absorption_events,
        where=hist_absorption_events > 0,
        out=np.zeros_like(counts_ejected, dtype=float)
    )
    
    # Plot results
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(
        bin_centers, yield_per_absorption,
        color='darkgreen', linewidth=2,
        label='Simulation (irregular spheres)'
    )
    ax.fill_between(
        bin_centers, yield_per_absorption,
        color='darkgreen', alpha=0.3, linewidth=0
    )
    
    # Add literature data if available
    try:
        Kimura_a10nm = pd.read_csv("literature-data/Fig2a-Kimura2016Solid10nm.csv")
        Kimura_bulk = pd.read_csv("literature-data/Fig2a-Kimura2016Bulk.csv")
        ax.plot(
            Kimura_a10nm["x"], Kimura_a10nm[" y"],
            linestyle='-.', color="blue", linewidth=2,
            label="Kimura 2016 (a=10nm)"
        )
        ax.plot(
            Kimura_bulk["x"], Kimura_bulk[" y"],
            linestyle='-.', color="orange", linewidth=2,
            label="Kimura 2016 (bulk)"
        )
    except FileNotFoundError:
        print("Literature data files not found. Skipping comparison plots.")
    
    ax.set_xlabel("Photon Energy (eV)")
    ax.set_ylabel("Yield Per Photoelectric Event")
    ax.set_xlim([5, 340])
    ax.set_ylim([1e-3, 1])
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title("Photoelectric Yield vs Literature")
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### Visualization: Photoelectron Trajectories

Visualize individual photon-to-photoelectron events through the geometry.

In [ ]:
# Create cross-section and plot sample events
if 'df_onlyphotoemission' in dir():
    section = geometry.section(
        plane_origin=geometry.centroid,
        plane_normal=[0, 1, 0]
    )
    plane_point = geometry.centroid
    verts = section.vertices
    
    V = verts - plane_point
    proj_x = V[:, 0]
    proj_y = V[:, 2]
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Identify and fill sphere sections
    for e in section.entities:
        pts = np.array([[proj_x[p], proj_y[p]] for p in e.points])
        if len(pts) < 3:
            continue
        if not np.allclose(pts[0], pts[-1]):
            pts = np.vstack([pts, pts[0:1]])
        
        try:
            poly = ShapelyPolygon(pts)
            if not poly.is_valid:
                poly = poly.buffer(0)
            if poly.is_empty or poly.area < 1e-8:
                continue
            
            circularity = 4 * np.pi * poly.area / (poly.length ** 2) if poly.length > 0 else 0
            if circularity > 0.7:
                coords = np.array(poly.exterior.coords)
                patch = Polygon(
                    coords, closed=True,
                    facecolor='lightgray', edgecolor='none', alpha=0.4
                )
                ax.add_patch(patch)
        except Exception:
            continue
    
    # Draw geometry outlines
    for e in section.entities:
        pts = e.points
        ax.plot(proj_x[pts], proj_y[pts], 'k-', lw=2, alpha=0.5)
    
    # Plot sample events
    sample_events = world_e_energy["Event_Number"].iloc[40:50]
    for idx, eventIN in enumerate(sample_events):
        eventdf = df_onlyphotoemission[df_onlyphotoemission["Event_Number"] == eventIN]
        gamma_eventdf = eventdf[eventdf["Particle_Type"] == "gamma"]
        e_eventdf = eventdf[eventdf["Particle_Type"] == "e-"]
        
        if len(gamma_eventdf) > 0:
            gamma_interwoven = np.empty((len(gamma_eventdf) * 2, 3))
            gamma_interwoven[0::2] = np.vstack(gamma_eventdf["Pre_Step_Position_mm"])
            gamma_interwoven[1::2] = np.vstack(gamma_eventdf["Post_Step_Position_mm"])
            gamma_interwoven = np.unique(gamma_interwoven, axis=0)
            ax.plot(
                gamma_interwoven[:, 1], gamma_interwoven[:, 2],
                '-', color='blue', alpha=0.8, lw=4,
                label='Photon' if idx == 0 else ''
            )
        
        if len(e_eventdf) > 0:
            e_interwoven = np.empty((len(e_eventdf) * 2, 3))
            e_interwoven[0::2] = np.vstack(e_eventdf["Pre_Step_Position_mm"])
            e_interwoven[1::2] = np.vstack(e_eventdf["Post_Step_Position_mm"])
            e_interwoven = np.unique(e_interwoven, axis=0)
            ax.plot(
                e_interwoven[:, 1], e_interwoven[:, 2],
                '-', color='red', alpha=0.8, lw=1,
                label='Electron' if idx == 0 else ''
            )
    
    ax.set_ylim([-0.17, 0.203])
    ax.set_xlim([-0.2, 0.2])
    ax.set_xlabel("Y position (mm)")
    ax.set_ylabel("Z position (mm)")
    ax.set_title("Sample Photoemission Events (Cross-section)")
    ax.set_aspect('equal')
    ax.legend()
    plt.tight_layout()
    plt.show()

### Face Illumination: Emitted Electrons

Visualize which faces of the geometry emit photoelectrons.

In [ ]:
# Face illumination for emitted electrons
if 'df_onlyphotoemission' in dir():
    print("Analyzing face illumination for emitted electrons...")
    surface, face_totals, _ = plot_face_illumination(
        world_e_energy, geometry,
        vmin=0, vmax=2
    )
    print(f"Emission per face range: {min(face_totals):.2f} to {max(face_totals):.2f}")
    
    surface_edited = surface.copy()
    surface_edited.unmerge_vertices()
surface_edited.show()

## Summary

This notebook has verified:
1. **Solar wind input distributions** - Electron and proton energy distributions match expected forms
2. **Photon input distribution** - Solar spectrum correctly sampled
3. **Photoelectron generation and emission** - Energy distributions and emission efficiency
4. **Comparison with literature** - Photoelectric yield from Geant4 vs literature data
5. **Geometric illumination** - Face-by-face particle illumination 

All results should be consistent with the physics models implemented in g4chargeit.